# PETs/TETs – Hyperledger Aries – Authority Agent (Issuing Authority) 🏛️


In [1]:
%%javascript
document.title='🏛️ Authority'

<IPython.core.display.Javascript object>

## PART 2: Issue a VC to the city Agents

**What:** Issue verifiable credentials (VCs) to all city

**Why:** City will be able to store VCs, and prove to relying parties that the agent is a city agency, without revealing their identity.

**How:** <br>
1. [Initiate Authority's AgentCommunicationManager (ACM)](#1) <br>
2. [Connect with City](#2)
3. [Issue VC to City](#3)

**Accompanying Agents and Notebooks:** 
* City 🏙️: `02_get_City_VC.ipynb`

---

### 0 - Setup
#### 0.1 - Imports

In [2]:
import os

from aries_cloudcontroller import AriesAgentController

import libs.helpers as helpers
from libs.agent_connection_manager import IssuingAuthority

#### 0.2 – Variables

In [3]:
# Get identifier data defined in notebook 00_init_authority_as_issuingAuthority.ipynb
identifiers = helpers.get_identifiers()
schema_city_id = identifiers["city_schema_identifiers"]["schema_id"]
cred_def_city_id = identifiers["city_schema_identifiers"]["cred_def"]

# Get environment variables
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

Get stored identifiers dictionary to access schema information 💾


---

<a id=1> </a>
### 1 – Initiate Authority Agent

#### 1.1 – Init ACA-PY agent controller

In [4]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

Initialising a controller with admin api at http://authority-agent:3021 and an api key of adminApiKey


#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [5]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.3 – Init ACM issuing authority

In [6]:
# The IssuingAuthority registers relevant webhook servers and event listeners
authority_agent = IssuingAuthority(agent_controller)

Successfully initiated AgentConnectionManager for a(n) Issuing Authority ACA-PY agent


---

<a id=2> </a>

### 2 – Establish a connection with the city
A connection with the credential issuer (i.e., the authority agent) must be established before a VC can be received. In this scenario, the City requests a connection with the Authority to be certified as an official city agency. Thus, the City agent sends an invitation to the Authority. In real life, the invitation can be shared via video call, phone call, or E-Mail. In this PoC, this is represented by copy and pasting the invitation into the city' notebooks.

### 2.1 – Receive invitation from `City` agent
Copy the invitation from Step 2.1 in the City's `01_hold_VC.ipynb` notebook into the following cell.
Several state changes of the connection between the city agent, the inviter (A), and the authority agent, the invitee (B),  are required before successfully establishing a connection:

| Step | State | Agent | Description | Function/Prompt/Variable |
| --- | --- | --- | --- | --- |
| 1 | invitation-sent | A | A sent an invitation to B | `create_connection_invitation()`
| 2 | invitation-received | B | B receives the invitation of A | Prompt: Paste invitation from A |
| 3 | request-sent | B | B sends B connection request | Prompt: Accept invitation OR `auto_accept=True` |
| 4 | request-received | A | A receives the connection request from B | Prompt: Accept invitation request response OR `auto_accept=True` |
| 5 | response-sent | A | A sends B response to B | - |
| 6 | response-received | B | B receives the response from A | - |
| 7 | active (completed) | A | B pings A to finalize connection | Prompt: Trust ping OR `auto_ping=True` |


In [7]:
# Setup for connection with Authority agent
alias = None
auto_accept = True # Accept response of Authority agent right away
auto_ping = True
public = False # Do not use public DID
multi_use = False # Invitation is only for one invitee

connection_id = authority_agent.create_connection_invitation(alias=alias, auto_accept=auto_accept, public=public, multi_use=multi_use, auto_ping=auto_ping)


Copy & paste invitation and share with external agent(s):
{
    '@id': '01fc690a-84e1-4e2b-ba24-e60d746e38db',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'Governance-Authority',
    'recipientKeys': ['7pQ9F6z8UPdSRVmZ61qxv3xTgaAbfVBYZcVYa6oRWLWE'],
    'serviceEndpoint': 'https://4d80-80-134-223-153.ngrok.io',
}

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  b4ac4c40-497b-45bf-bca5-9c942fcb21cf
State :  invitation (invitation-sent)
Routing State : none
Their Role :  invitee
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  b4ac4c40-497b-45bf-bca5-9c942fcb21cf
State :  request (request-received)
Routing State : none
Connection with :  City-Agency
Their Role :  invitee
---------------

<div style="font-size: 25px"><center><b>Break Point 1</b></center></div>
<div style="font-size: 50px"><center>🏛 ➡️ 🏙️</center></div><br>
<center><b>Please return to the City's notebook 🏙️. <br>Check the prompts in Step 2.1 (e.g., if auto_accept or auto_ping are set to False), and then proceed to Step 3.</b></center>

---

<a id=3> </a>
### 3 – Issue City a VC
#### 3.1 – Send City a request to fill out required information

In [8]:
# Contact authority agent
basic_message = "Hello City. I would like to certify you as an official city agency. Please tell me your city and country."
authority_agent.send_message(connection_id, basic_message)

Sent message via Connection ID b4ac4c40-497b-45bf-bca5-9c942fcb21cf


<div style="font-size: 25px"><center><b>Break Point 3</b></center></div>
<div style="font-size: 50px"><center>🏛 ➡️ 🏙️</center></div><br>
<center><b>Please return to the City's notebook 🏙️. <br>Continue with Step 3.1</b></center>

#### 3.2 – See response of City agent

In [9]:
# Verify inbox
message_ids = authority_agent.verify_inbox()

for m_id in message_ids:
    authority_agent.get_message(m_id)


---------------------------------------------------------------------
Message Inbox
> 1 Message(s) via Connection ID b4ac4c40-497b-45bf-bca5-9c942fcb21cf:
	 * Message ID :  d3b54c06-be86-47fc-b835-0d9204d51095
---------------------------------------------------------------------

---------------------------------------------------------------------
Message received
Connection ID :  b4ac4c40-497b-45bf-bca5-9c942fcb21cf
Message ID :  d3b54c06-be86-47fc-b835-0d9204d51095
State :  received
Time :  2021-08-24 11:40:31.332256Z
Text :  {"city": "Berlin", "country": "DE"}
---------------------------------------------------------------------


### 3.3 – Offer VC to `City` agent
The next step is to offer a VC to the city agent. The city can then request the offer and store it in their wallet. The following table provides an overview of the individual states between I (Issuer, the Authority agent) and H (Holder, the city).

| Step | State | Role | Description | Function/Prompt/Variable |
| --- | --- | --- | --- | --- |
| 1 | offer_sent | I | I sends I VC offer with personalized information to H| `offer_vc()` |
| 2 | offer_received | H | H receives offer made by I | - |
| 3 | request_sent | H | Request VC offer | `request_vc()` AND (Prompt: request VC OR `auto_request=True`) |
| 4 | request_received | I | M1's request to get VC was received | - |
| 5 | credential_issued | I | Automatic response to issue credential | - |
| 6 | credential_received | H| H receives VC and is asked to store it | Prompt: Store VC OR `auto_store=True`
| 7 | credential_acked | I / H | Credential was issued and stored | - |

In [10]:
# MAKE VC ZKP-able! SEE https://github.com/hyperledger/aries-cloudagent-python/blob/main/JsonLdCredentials.md
comment = "Issuing VC that City is a city agency"
auto_remove = True
trace = False

# Offer City a VC with city_schema
authority_agent.offer_vc(
    connection_id, 
    schema_city_id,
    cred_def_city_id,
    comment=comment, 
    # Comment out next line if you do not want to get the prompts to enter VC information
    #credential_attributes=[{"name": "manufacturerName", "value": "undisclosedCity"}, {"name": "manufacturerCity", "value": "Berlin"}, {"name": "manufacturerCountry", "value": "Germany"}, {"name": "isManufacturer", "value": "TRUE"}]
)

Please enter the following information for the certify-city-agency scheme: 


city:  Berlin
isCityAgency:  TRUE
country:  German
Is the information correct? [yes/no] no


Please enter the information again.
Please enter the following information for the certify-city-agency scheme: 


city:  Berlin
isCityAgency:  TRUE
country:  DE
Is the information correct? [yes/no] yes



---------------------------------------------------------------------
Handle Issue Credential Webhook: Issue Credential Handler
Connection ID : b4ac4c40-497b-45bf-bca5-9c942fcb21cf
Credential exchange ID : 217d916a-9401-4c56-b00c-9dc05bfdcf37
Agent Protocol Role :  issuer
Protocol State :  offer_sent
---------------------------------------------------------------------

Proposed Credential : 
{
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview',
    'attributes': [
        {'name': 'city', 'value': 'Berlin'},
        {'name': 'isCityAgency', 'value': 'TRUE'},
        {'name': 'country', 'value': 'DE'},
    ],
}

---------------------------------------------------------------------
Handle Issue Credential Webhook: Issue Credential Handler
Connection ID : b4ac4c40-497b-45bf-bca5-9c942fcb21cf
Credential exchange ID : 217d916a-9401-4c56-b00c-9dc05bfdcf37
Agent Protocol Role :  issuer
Protocol State :  request_received
-------------------------------

<div style="font-size: 25px"><center><b>Break Point 5</b></center></div>
<div style="font-size: 50px"><center>🏛 ➡️ 🏙️</center></div><br>
<center><b>Please return to the City's notebook 🏙️. <br>Continue with Step 3.3</b></center>

---

## 4 - Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [11]:
await agent_controller.terminate()

--- 

### 🔥🔥🔥 You are done 🙌 and can close this notebook now 🔥🔥🔥
